In [1]:
from mysql import connector

**Steps :**

`Download MySql Workbench ` [Click Here](https://dev.mysql.com/downloads/workbench/)

`Open MySql Workbench and Open MySql connector`

`On MyConnections click (+) symbol to create new connection`

`Add username and click on` **store in password vault** `for password`

### Connecting MySql with our Credentials..

> `Use those crediantials here to connect..`

In [2]:
Conn = connector.connect(host="localhost",
                        user="root",
                        password="root")

In [3]:
Conn

### Creating Cursor Object for Executing Sql Commands

In [4]:
cursor = Conn.cursor()

In [7]:
cursor.execute("SHOW DATABASES")

for x in cursor:
    print(x)

('information_schema',)
('mysql',)
('new_schema',)
('performance_schema',)


### By Default there are some databases, to create customdatabase use sql command `CREATE DATABASE dbname`

In [8]:
cursor.execute("CREATE DATABASE drugs_db")

In [9]:
cursor.execute("SHOW DATABASES")

for x in cursor:
    print(x)

('information_schema',)
('drugs_db',)
('mysql',)
('new_schema',)
('performance_schema',)


`Here we can see our database tc_db`

In [10]:
cursor.execute("USE drugs_db")

`Above we are Selecting the created database as the active one`

In [11]:
cursor.execute('''CREATE TABLE concept(concept_id INT(11) NOT NULL, 
                                    concept_name VARCHAR(255) NOT NULL, 
                                    domain_id VARCHAR(20) NOT NULL, 
                                    vocabulary_id VARCHAR(20) NOT NULL, 
                                    concept_class_id VARCHAR(20) NOT NULL,
                                    standard_concept VARCHAR(1) NULL DEFAULT NULL, 
                                    concept_code VARCHAR(50) NOT NULL,
                                    valid_start_date DATE NOT NULL,
                                    valid_end_date DATE NOT NULL,
                                    invalid_reason VARCHAR(1) NULL DEFAULT NULL);''')

In [12]:
cursor.execute("SHOW TABLES")

for x in cursor:
    print(x)

('concept',)


`Here we got our concept table.Let's start creating all other remaining tables....`

In [13]:
cursor.execute('''CREATE TABLE vocabulary (vocabulary_id VARCHAR(20) NOT NULL,
                                           vocabulary_name VARCHAR(255) NOT NULL, 
                                           vocabulary_reference VARCHAR(255) NULL DEFAULT NULL, 
                                           vocabulary_version VARCHAR(255) NULL DEFAULT NULL, 
                                           vocabulary_concept_id INT(11) NOT NULL);''')

In [14]:
cursor.execute('''CREATE TABLE standard_case_drug (primaryid VARCHAR(512) NULL DEFAULT NULL, 
                                                   isr VARCHAR(512) NULL DEFAULT NULL, 
                                                   drug_seq VARCHAR(512) NULL DEFAULT NULL, 
                                                   role_cod VARCHAR(512) NULL DEFAULT NULL, 
                                                   standard_concept_id INT(11) NULL DEFAULT NULL);''')

In [15]:
cursor.execute('''CREATE TABLE standard_case_indication (primaryid VARCHAR(512) NULL DEFAULT NULL, 
                                                        isr VARCHAR(512) NULL DEFAULT NULL, 
                                                        indi_drug_seq VARCHAR(512) NULL DEFAULT NULL, 
                                                        indi_pt VARCHAR(512) NULL DEFAULT NULL, 
                                                        indication_concept_id INT(11) NULL DEFAULT NULL, 
                                                        snomed_indication_concept_id INT(11) NULL DEFAULT NULL);''')

In [16]:
cursor.execute('''CREATE TABLE standard_case_outcome (primaryid VARCHAR(512) NULL DEFAULT NULL, 
                                                      isr VARCHAR(512) NULL DEFAULT NULL, 
                                                      pt VARCHAR(512) NULL DEFAULT NULL, 
                                                      outcome_concept_id INT(11) NULL DEFAULT NULL,
                                                      snomed_outcome_concept_id INT(11) NULL DEFAULT NULL);''')

In [17]:
cursor.execute('''CREATE TABLE standard_case_outcome_category (primaryid VARCHAR(512) NULL DEFAULT NULL, 
                                                               isr VARCHAR(512) NULL DEFAULT NULL,
                                                               outc_code VARCHAR(512) NULL DEFAULT NULL, 
                                                               snomed_concept_id INT(11) NULL DEFAULT NULL);''')

In [18]:
cursor.execute('''CREATE TABLE standard_drug_outcome_contingency_table (drug_concept_id INT(11) NULL DEFAULT NULL, 
                                                                        outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                                        count_a BIGINT(20) NULL DEFAULT NULL, 
                                                                        count_b DECIMAL(10,0) NULL DEFAULT NULL, 
                                                                        count_c DECIMAL(10,0) NULL DEFAULT NULL, 
                                                                        count_d DECIMAL(10,0) NULL DEFAULT NULL);''')

In [19]:
cursor.execute('''CREATE TABLE standard_drug_outcome_count (drug_concept_id INT(11) NULL DEFAULT NULL, 
                                                            outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                            drug_outcome_pair_count BIGINT(20) NULL DEFAULT NULL, 
                                                            snomed_outcome_concept_id INT(11) NULL DEFAULT NULL);''')

In [20]:
cursor.execute('''CREATE TABLE standard_drug_outcome_drilldown (drug_concept_id INT(11) NULL DEFAULT NULL, 
                                                                outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                                snomed_outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                                primaryid VARCHAR(512) NULL DEFAULT NULL, 
                                                                isr VARCHAR(512) NULL DEFAULT NULL);''')

In [21]:
cursor.execute('''CREATE TABLE standard_drug_outcome_statistics (drug_concept_id INT(11) NULL DEFAULT NULL, 
                                                    outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                    snomed_outcome_concept_id INT(11) NULL DEFAULT NULL, 
                                                    case_count BIGINT(20) NULL DEFAULT NULL, 
                                                    prr DECIMAL(20,5) NULL DEFAULT NULL, 
                                                    prr_95_percent_upper_confidence_limit DECIMAL(20,5) NULL DEFAULT NULL, 
                                                    prr_95_percent_lower_confidence_limit DECIMAL(20,5) NULL DEFAULT NULL, 
                                                    ror DECIMAL(20,5) NULL DEFAULT NULL, 
                                                    ror_95_percent_upper_confidence_limit DECIMAL(20,5) NULL DEFAULT NULL,
                                                    ror_95_percent_lower_confidence_limit DECIMAL(20,5) NULL DEFAULT NULL);''')

In [22]:
cursor.execute('''CREATE TABLE standard_unique_all_case (caseid VARCHAR(512) NULL DEFAULT NULL,
                                                        primaryid VARCHAR(512) NULL DEFAULT NULL,
                                                        isr VARCHAR(512) NULL DEFAULT NULL);''')

`Total 11 tables were created....`

In [23]:
cursor.execute("SHOW TABLES")

for table in cursor:
    print(table)

('concept',)
('standard_case_drug',)
('standard_case_indication',)
('standard_case_outcome',)
('standard_case_outcome_category',)
('standard_drug_outcome_contingency_table',)
('standard_drug_outcome_count',)
('standard_drug_outcome_drilldown',)
('standard_drug_outcome_statistics',)
('standard_unique_all_case',)
('vocabulary',)


### **AEOLUS Data Loading into Sql**

In [24]:
Conn = connector.connect(host="localhost",
                        user="root",
                        password="root",
                        database="drugs_db",
                        allow_local_infile=True)      # Note :  Without this line may face lots of issues.Most Important..

cursor = Conn.cursor()

In [25]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/concept.tsv' INTO TABLE concept FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [26]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/vocabulary.tsv' INTO TABLE vocabulary FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [27]:
cursor.execute('SELECT * FROM VOCABULARY')

for records in cursor:
    print(records)

('ABMS', 'Provider Specialty (American Board of Medical Specialties)', 'http://www.abms.org/member-boards/specialty-subspecialty-certificates', '', 45756746)
('APC', 'Ambulatory Payment Classification (CMS)', 'http://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/HospitalOutpatientPPS/Hospital-Outpatient-Regulations-and-Notices.html', '2011-28-09', 44819132)
('ATC', 'WHO Anatomic Therapeutic Chemical Classification', 'FDB UK distribution package', 'RxNorm Full 20150706', 44819117)
('CCS', 'Clinical Classifications Software for ICD-9-CM (HCUP)', 'http://www.hcup-us.ahrq.gov/toolssoftware/ccs/ccs.jsp#download', 'Not implemented yet', 44819142)
('CIEL', 'Columbia International eHealth Laboratory (Columbia University)', 'https://wiki.openmrs.org/display/docs/Getting+and+Using+the+MVP-CIEL+Concept+Dictionary', 'Openmrs 1.11.0 20150227', 45905710)
('Cohort', 'Legacy OMOP HOI or DOI cohort', 'OMOP generated', '', 44819123)
('Cohort Type', 'OMOP Cohort Type', 'OMOP generated', '', 44819

In [28]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_case_drug.tsv' INTO TABLE standard_case_drug 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [29]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_case_indication.tsv' INTO TABLE standard_case_indication 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [30]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_case_outcome.tsv' INTO TABLE standard_case_outcome 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')


In [31]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_case_outcome_category.tsv' INTO TABLE standard_case_outcome_category 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [32]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_drug_outcome_contingency_table.tsv' INTO TABLE standard_drug_outcome_contingency_table 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [33]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_drug_outcome_count.tsv' INTO TABLE standard_drug_outcome_count 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [34]:
cursor.execute('''LOAD DATA LOCAL INFILE'aeolus_v1/standard_drug_outcome_drilldown.tsv' INTO TABLE standard_drug_outcome_drilldown 
                FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')

In [35]:
cursor.execute('''LOAD DATA LOCAL INFILE 'aeolus_v1/standard_drug_outcome_statistics.tsv' INTO TABLE standard_drug_outcome_statistics 
                    FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' IGNORE 0 LINES;''')